# Permuterm Index

### Made By Yuliia Hetman

In [1]:
%%time
from multiprocessing import Pool
import re, os, json
from string import digits
import pandas as pd

docIDs = list(range(0, 18))


def isUkrainian(word):
    ukrainian = [96, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217]
    for l in word:
        if ord(l) not in ukrainian:
            return False
    return True

def check_vocabulary(vocabulary):
        return [k + '$' for k in set(vocabulary) if isUkrainian(k) == True]
        
def delete_sings_digits(content):
    punc = '''!()-'”№[]{};:"\,“«»<>./?@#$%—…^&*_~|–abcdefghijklmnoqprstuvwxyz'''
    content = "".join([ele for ele in content.lower() if ele not in punc])
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab

def read_book(book):
    dirname='../books/'
    with open(dirname + book, 'r', encoding='utf-8') as file:
        text = create_vocabulary(file.read())
    return text

        
def get_vocabulary(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    with Pool(8) as p:
        vocabul = p.map(read_book, books)
    for i, book in enumerate(books):
        dictionary[book.replace('.txt', '')] = vocabul[i]
        vocabulary += vocabul[i]
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = get_vocabulary()
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')

Total number of words : 708449
Total number of tokens : 94292
CPU times: user 640 ms, sys: 394 ms, total: 1.03 s
Wall time: 1.31 s


In [2]:
vocabulary = check_vocabulary(vocabulary)
vocabulary[:5]

['грубший$', 'молоді$', 'кармазинова$', 'славніший$', 'помічаєш$']

In [3]:
%%time
import itertools
def create_indexes(word):
    arr = list()
    length = len(word)
    for i in range(length):
        arr.append(word[i:length] + word[:i])
    return arr
        
def create_permuterm(vocabulary):
    permuterm = list()
    with Pool(16) as p:
        permuterm = p.map(create_indexes, vocabulary)
    all_permterms = list(itertools.chain.from_iterable(permuterm))
    return all_permterms

res = create_permuterm(vocabulary)
res[:10]

CPU times: user 491 ms, sys: 150 ms, total: 641 ms
Wall time: 796 ms


In [4]:
%% time

def compare_particles(term, parts, temp=0):
    for i in range(len(parts)):
        ind = term.find(parts[i])
        if temp < ind:
            temp = ind
            continue
        else:
            return False
    return True


def find(word, permuterms, parts):
    similar = list()
    for term in permuterms:
        if word == term[:len(word)] and compare_particles(term, parts, temp=len(word) - 1) == True:
            similar.append(term)
        else:
            continue
    return similar

def post_process_terms(terms):
    beautify = list()
    for term in terms:
        sign = term.find('$')
        beautify.append(term[sign + 1:] + term[:sign])
    return beautify

def search_word(string, permuterms):
    string += '$'
    ind = string.rfind('*')
    jockers = string.count('*')
    to_find = string[ind + 1:len(string)] + string[:string.find('*')]
    if jockers > 1:
        parts = string.split('*')
        parts = parts[1:-1]
    similar_terms = find(to_find, permuterms, parts)
    beautify = post_process_terms(similar_terms)
    return beautify
    
    


search_word("на*ри*ва*є", res)

UsageError: Cell magic `%%` not found.
